# Reto 03-A-Grounding (Fundamentación) 

## 1. Descripción General 

La Fundamentación (Grounding) es una técnica utilizada cuando deseas que el modelo devuelva respuestas fiables a una pregunta dada. A menudo, los modelos GPT necesitarán contexto adicional para proporcionar una respuesta que no alucine, también conocida como dar respuestas falsas. Recuerda que estos modelos GPT solo han sido entrenados con datos hasta septiembre de 2021. Además, los modelos no han sido entrenados con datos específicos de casos de uso.

Existen un par de métodos para llevar a cabo la fundamentación. En este escenario, nos centraremos principalmente en el grounding básico dentro del prompt. En el reto cuatro, verás otras aplicaciones de fundamentación utilizando una base de conocimientos externa e implementando la técnica de Generación Aumentada con Recuperación, o RAG.

Para entender los conceptos básicos de la fundamentación y sus beneficios, este notebook te guiará a través de un ejemplo. A continuación se muestra el escenario que incorporaremos.

## 2. Escenario

Estás escribiendo un informe sobre el torneo de tenis de Wimbledon y necesitas discutir el último partido. Descubre quién fue el ganador de los individuales masculinos y femeninos en 2023.

## 3. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos clave de configuración que completaste en las tareas anteriores.

In [1]:
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y tus claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

model=os.getenv("CHAT_MODEL_NAME")

### 3.0 Funciones Auxiliares

**get_completion** ayuda a crear una respuesta de OpenAI utilizando el modelo de completado de texto de tu elección.

In [3]:
def get_chat_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

### 3.1 Antes de la Fundamentación

#### Tarea #1 del Estudiante:

Edita el prompt en la celda siguiente para hacerle una pregunta al modelo sobre el escenario.

Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. Find out who the 2023 winner was for the mens and womens singles.

In [7]:
prompt = f"""
Who was the winner of election for president  in USA at 2024?
"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")


Response: As an AI language model, I cannot predict future events or election results.



¿Es esta la respuesta que esperabas?

### 3.2 Después de la Fundamentación

#### Tarea #2 del Estudiante:

Modifica el prompt a continuación para fundamentar el modelo. ¿Cómo puedes obtener una respuesta más precisa que la que recibiste anteriormente?

In [8]:
context = f"""Las Elecciones presidenciales de Estados Unidos de 2024, celebradas el martes 5 de noviembre de 2024, fueron las sexagésimas elecciones presidenciales en dicho país.1​La fórmula del Partido Republicano liderada por Donald Trump y James Vance le ganó en el Colegio Electoral a la fórmula del Partido Demócrata liderada por Kamala Harris y Tim Walz.

El límite establecido por la Vigesimosegunda Enmienda constitucional permitió al expresidente Donald J. Trump ser reelecto para un segundo período de gobierno. Las elecciones primarias presidenciales y asambleas electorales de ambos partidos se llevaron a cabo durante los primeros seis meses de 2024.

Las elecciones presidenciales coincidieron con las Elecciones al Senado de los Estados Unidos de 2024 en las que se eligieron a 33 senadores y con las Elecciones a la Cámara de Representantes de los Estados Unidos de 2024 en las que se eligió la totalidad de la Cámara de Representantes. En las tres elecciones se reflejó el aumento de votos hacia el Partido Republicano, logrando este partido el control de ambas cámaras además de la presidencia.

El actual presidente, Joe Biden, miembro del Partido Demócrata, inicialmente se postuló para la reelección y se convirtió en el candidato presunto del partido. Sin embargo, debido a preocupaciones sobre su salud y su edad avanzada, se retiró de la carrera el 21 de julio y respaldó a la vicepresidenta Kamala Harris, quien lanzó su campaña presidencial ese mismo día. La retirada de Biden de la carrera lo convierte en el primer presidente que no se presenta a la reelección desde Lyndon B. Johnson en 1968.

El predecesor de Biden, el expresidente Donald Trump, se postuló por tercera vez como candidato presidencial, después de haber perdido ante Biden en las elecciones presidenciales de 2020. Se convirtió en el primer político en ser candidato presidencial del Partido Republicano durante tres veces consecutivas. Aunque Richard Nixon fue candidato por tercera vez en 1972, sus candidaturas no fueron consecutivas, ya que las anteriores habían sido en 1960 y 1968.2​ En mayo de 2024, Trump se convirtió en la primera persona que haya ejercido la presidencia de Estados Unidos, en ser condenada por un delito, y en el primer político perteneciente a alguno de los dos partidos políticos principales, en ser candidato presidencial con una condena penal.3​

El 6 de noviembre, Donald Trump logró ganar en los estados pendulares necesarios, además de contar con una ventaja en el voto popular por primera vez,4​5​ siendo el primer republicano en dos décadas en lograrlo (el último siendo George W. Bush en 2004),6​7​ además de ser reelecto no consecutivamente como Presidente de los Estados Unidos (el segundo tras Grover Cleveland en 1892),8​9​ venciendo a la vicepresidente incumbente, Kamala Harris.10​11​12​13​14​15​16​17​18​19​ El voto masculino, incluyendo el de los varones latinos, jugó un papel importante en su favor.20​21​ Trump recibió mensajes de felicitación de políticos de todo el mundo.22​

Con sus 312 votos electorales obtenidos Trump obtuvo el mejor resultado en el colegio electoral para un republicano desde George H. W. Bush en 1988.23​"""

prompt = f"""
Who was the winner of election for president  in USA at 2024?

this is the context: {context}

"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")

Response: The winner of the election for president in the USA in 2024 was Donald Trump, who was reelected for a second term with his running mate James Vance. They defeated the Democratic Party's Kamala Harris and Tim Walz in the Electoral College. Trump became the first president to be reelected after being impeached and convicted of a crime. Joe Biden, the incumbent president, initially ran for reelection but withdrew from the race due to concerns about his health and age. Trump's victory was aided by strong support from male voters, including Latino men. He received congratulations from politicians around the world and won 312 electoral votes, the best result for a Republican since George H.W. Bush in 1988.



¿Es esta la respuesta que esperabas para ayudarte a escribir tu informe?

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra que has comprendido cómo fundamentar un modelo y por qué es importante.
* Asegúrate de obtener una respuesta precisa a tu pregunta que te ayudará a completar el escenario descrito al principio del desafío.
